In [59]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("postoffice.csv")

columns_to_remove = ['duration', 'alt_name', 'discontinued', 'county1', 'county2', 'county3',  'id', 'gnis_name', 'gnis_match', 'gnis_county', 'gnis_state', 'gnis_feature_id', 'gnis_feature_class', 'gnis_orig_name', 'gnis_orig_county', 'gnis_latitude', 'gnis_longitude', 'gnis_elev_in_m', 'gnis_dist']

# Remove the specified columns
df_cleaned = df.drop(columns=columns_to_remove)
df_cleaned = df_cleaned.dropna(subset=['latitude', 'longitude', 'established'])


# Keep only the current running offices
#df_cleaned = df_cleaned[df_cleaned['duration'].isna()]

# Calculate 'duration' based on 'established' column if it's missing
#df_cleaned['duration'] = df_cleaned.apply(lambda row: (2021 - row['established']) if pd.isnull(row['duration']) else row['duration'], axis=1)

print(df_cleaned)

# Check for non-convertible values in 'established' column
non_convertible_established = df_cleaned[df_cleaned['established'].isnull()]['established']
print("Non-convertible values in 'established' column:")




df_cleaned_sorted = df_cleaned.sort_values(by='established', ascending=False)

# Keep only the top 1000 recent post offices
#df_top_1000_recent = df_cleaned_sorted.head(1000)

df_top_1000_recent = df_cleaned_sorted

# Check the first few rows of the resulting DataFrame
print(df_top_1000_recent.head())


                 name      orig_name state orig_county  established  \
2               AARON          AARON    MO       Bates       1895.0   
5               AARON          AARON    GA     Bulloch       1909.0   
7               AARON          AARON    SC    Anderson       1892.0   
8          AARONSBURG  AARONSBURG(H)    PA      Centre       1792.0   
11             ABADYL         ABADYL    MO   Christian       1895.0   
...               ...            ...   ...         ...          ...   
166129         ZURICH         ZURICH    MT      Blaine       1907.0   
166131           ZYBA           ZYBA    KS      Sumner       1887.0   
166133         ZYBACH         ZYBACH    TX     Wheeler       1910.0   
166136           TRUE           TRUE    TX       Young       1894.0   
166137  SIERRA BLANCA  SIERRA BLANCA    TX    Hudspeth       1882.0   

        continuous  stamp_index  coordinates   latitude   longitude  
2             True          2.0         True  38.422222  -94.154167  
5      

In [62]:

# Convert 'duration' and 'established' columns to integers
#df_top_1000_recent['duration'] = df_top_1000_recent['duration'].astype(int)
df_top_1000_recent['established'] = df_top_1000_recent['established'].astype(int)


# Write the cleaned data to a new CSV file
df_top_1000_recent.to_csv("postoffice_cleaned.csv", index=False)

print("Rows with NA values removed. Cleaned data saved to postoffice_cleaned.csv")


Rows with NA values removed. Cleaned data saved to postoffice_cleaned.csv


In [64]:
import pandas as pd

# Read the cleaned CSV file
df_cleaned = pd.read_csv("postoffice_cleaned.csv")

# Dictionary mapping short names to long names of states
state_names_long = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'
}

# Invert the dictionary to get short names as keys and long names as values
state_names_short = {v: k for k, v in state_names_long.items()}

# Map short state names to long state names
df_cleaned['state_longform'] = df_cleaned['state'].map(state_names_long)

# Aggregate number of post offices per state
state_post_offices_long = df_cleaned.groupby('state_longform').size().reset_index(name='num_post_offices')

# Map long state names back to short state names
state_post_offices_long['state'] = state_post_offices_long['state_longform'].map(state_names_short)

# Concatenate short and long state names with aggregated number of post offices
state_post_offices = pd.concat([state_post_offices_long[['state', 'state_longform', 'num_post_offices']], state_post_offices_long[['state', 'state_longform', 'num_post_offices']]])

# Calculate the centroid of each state and save it to the CSV
centroid_latitudes = []
centroid_longitudes = []

for state_longform in state_post_offices['state_longform']:
    state_data = df_cleaned[df_cleaned['state_longform'] == state_longform]
    centroid_latitude = state_data['latitude'].mean()
    centroid_longitude = state_data['longitude'].mean()
    centroid_latitudes.append(centroid_latitude)
    centroid_longitudes.append(centroid_longitude)

state_post_offices['centroid_latitude'] = centroid_latitudes
state_post_offices['centroid_longitude'] = centroid_longitudes

# Write the aggregated data to a new CSV file
state_post_offices.to_csv("statewise_post_offices.csv", index=False)

print("State-wise post office data saved to statewise_post_offices.csv")


State-wise post office data saved to statewise_post_offices.csv
